# Cluster Analysis with K-means (Text Clustering)

<img style="float: l;" src="./images/k-means-clustering.png" width="500px">

## Environment Setup

#### First we will import some Python packages that we will use.

In [1]:
import nltk
import re
import pandas as pd
import pickle as pkl
from string import punctuation
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import MDS
%matplotlib inline 
import matplotlib.pyplot as plt


## More Environment Setup

#### We may also need to pull in some nltk resources. These resources will help us clean the data.

In [2]:
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jpferraro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jpferraro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Syntatic NLP Processing

#### We will define some Python functions that will perform some syntatic work on our corpus. 

In [3]:
# This function will remove unnecessay puncuation that can add to noise
def stripPunctuation(text):
    return ''.join(c for c in text if c not in punctuation)

# We will want to eventually tokenize a document into sentences and words
def tokenize(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out tokens not containing letters (e.g., numeric tokens, some punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

# We cache the stopword upfront for perfomance
cachedStopWords = stopwords.words("english")

# We will want to remove stopwords which are typiccially high frequency words that aid with  
# text fluency but do not provide much information gain.
def removeStopwords(text):
    no_stop = ' '.join([word for word in text.split() if word not in cachedStopWords])
    return no_stop

## Retrieving our Corpus

#### Let's pull in our corpus that we had serialized out to disk.  

In [4]:
#file = open('differential-corpus.pkl','rb')
file = open('pna-corpus.pkl','rb')
corpus = pkl.load(file)
file.close()

##  Preparing our Corpus for Processing

#### First step is to clean up our corpus. We will strip pucntuation, remove stopwords, and tokenize our text. Actually removing stopwords and tokinization can be done by a downstream function as you will see. We do it here on the first document in our corpus just so you can get a good sense how the corpus gets transformed.

In [5]:
cleanedCorpus = []
ranks = []
labels = []
idx = 1
for key in corpus:        
    cleanedText = stripPunctuation(corpus[key])
    if (idx < 2):
        print("**** ORIGINAL CLINICAL DOCUMENT ****")
        print()
        print(corpus[key])
        print("**** SOME PUNCTUATION REMOVED ****")
        print()        
        print(cleanedText)
        print("**** STOP WORDS REMOVED ****")
        print()
        textWithoutStopwords = removeStopwords(cleanedText)
        print(textWithoutStopwords)
        tokenizedText = tokenize(cleanedText)
        print()
        print("**** LOWER CASE AND TOKENIZE ****")
        print()
        print(tokenizedText)
        print()
    cleanedCorpus.append(cleanedText)
    labels.append(key)
    ranks.append(idx)   
    idx += 1

**** ORIGINAL CLINICAL DOCUMENT ****




     DATE: [**2996-12-2**] 10:25 AM
     CT CHEST W/O CONTRAST                                           Clip # [**Clip Number (Radiology) 11781**]
     Reason: please eval for loculated pleural effusion on the right and 
     Field of view: 30
     ______________________________________________________________________________
     UNDERLYING MEDICAL CONDITION:
      73 year old woman with h/o RUL subtotal resection for lung CA (3/'[**95**]) as well
      as a LLL superior segment resection  for Tb and now presents w/ a RLL PNA and
      parapneumonic effusion concerning for empyema
     REASON FOR THIS EXAMINATION:
      please eval for loculated pleural effusion on the right and place a pigtail
      catheter (send specimen for Gm stain C+S, AFB smear, cytology, Fungal culture)
      please do ASAP as pt may require to go to the OR later this afternoon
     ______________________________________________________________________________
        

##  Quantifying our Clinical Corpus for Cluster Analysis

#### We need to somehow quantify the importance of the words found in a document relative to one another, as well as, relative to the entire corpus. Simply using frequency counts of words are a bias measurement. Longer documents naturally will have greater term frequencies than shorter documents. 

#### We can borrow from the field of informatin retrieval and use a measurement known as tf-idf, short for term frequency-inverse document frequency. This statistic is intended to reflect how important a word is in a document relative to the other words found in the document, as well as, relative to how often a word may be used across a corpus. 

#### TfidfVectorizer will not only produce our tf-idf word statistics but it will also first remove less important words known as stopwords and tokenize our corpus. The end result is a sparse matrix where each row represents a document in our corpus and each column is a distinct word found across all of the documents. 

#### 1) You should be able to understand why we want this information represented as a sparse matrix. Why do you think? 


In [6]:
# Note that max-df is set such that we keep words that can appear across 100% of the corpus and min-df is set
# such that we throw away words that are in less than 10% of our document corpus. We will also generate 
# unigrams and bigrams which is about right form clinical notes. Beyond bigrams we get very sparse....
tfidfVectorizer = TfidfVectorizer(max_df=1.0, max_features=200000,
                                  min_df=0.1, stop_words='english',
                                  use_idf=True, tokenizer=tokenize, ngram_range=(1,2))   

tfidfMatrix = tfidfVectorizer.fit_transform(cleanedCorpus)

print("**** STORED AS A SPARSE MATRIX ****")
print(tfidfMatrix.shape)
print()

**** STORED AS A SPARSE MATRIX ****
(11108, 222)



##  Generated Data to Keep

#### We want to hold onto the terms that were selected by our tf-idf vectorizer. 

#### We will also perform some high-dimensional geometry by calculating the cosine angel between multi-dimensional vectors that represent the text documents. Each element in the document vector represents a distint word that was used in the document. This word element is represented as a tf-idf numeric value. The cosine angle between two document vectors can be measured to determine how alike in topic two documents may be. We will calculate the pairwise similarity between all document pairs. We will need these distance measures for later when we try to visualize our clusters. 


In [7]:
distinctTerms = tfidfVectorizer.get_feature_names()
print("**** SOME DISTINCT TERMS ***")
print(distinctTerms[-10:])
print(distinctTerms[:10])
print()
dist = 1 - cosine_similarity(tfidfMatrix)

**** SOME DISTINCT TERMS ***
['vein', 'venous', 'view', 'visualized', 'wall', 'wo', 'wo contrast', 'woman', 'year', 'year old']
['abdomen', 'abdominal', 'acute', 'admitting', 'admitting diagnosis', 'air', 'amt', 'amt final', 'amt underlying', 'aorta']



##  Time to Build Clusters

#### We will use K-means clustering to search for symantic patterns within our corpus.  K-means is a vector quantization method that attemps to partition $N$ observations (our clinical documents) into $K$ clusters. It assigns an observation to a cluster by trying to allign the observations with the cluster centroid. The cluster centroid represents the center of the cluster. K-means tries to minimize the within-cluster-variance which is a measure of compactness. So the idea is to generate as compact clusters as possible given a set of observations to assign.  

#### K-means is known as a hard-clustering method because each observations must be assigned to a single cluster. Other clustering methods such as expectation-maximization are known as soft-clustering methods because an observation can belong to multiple clusters with a certain probability. 

#### With K-means we need to determine the number of clusters we will produce aprioi. There is no magic here, the best approach is performing some trial and error. 


In [ ]:
numClusters = 10
km = KMeans(n_clusters=numClusters)
km.fit(tfidfMatrix)
clusters = km.labels_.tolist()

## Some Basic Cluster Statistics

#### Let's look at the number of documents that were assigned to each cluster. We can also take a look at the group mean of the cluster althogh it does not provide a great deal of information.

In [9]:
clinicalDocuments = { 'labels': labels, 'rank': ranks, 'corpus': cleanedCorpus, 'cluster': clusters }
frame = pd.DataFrame(clinicalDocuments, index = [clusters] , columns = ['rank', 'labels', 'corpus', 'cluster'])
print("**** CLUSTER COUNTS ****")
print(frame['cluster'].value_counts())
print()
grouped = frame['rank'].groupby(frame['cluster'])
print("**** CLUSTER MEANS ****")
print(grouped.mean())
print()


**** CLUSTER COUNTS ****
6    1994
4    1752
7    1436
0    1395
8    1222
2    1071
1     882
9     666
5     350
3     340
Name: cluster, dtype: int64

**** CLUSTER MEANS ****
cluster
0    5294.246595
1    5553.170068
2    5382.037348
3    5497.373529
4    5876.120434
5    6300.640000
6    5118.392678
7    5551.051532
8    5601.653028
9    6396.328829
Name: rank, dtype: float64



## Better Insight into our Clusters

#### We will obviously want to understand what documents clustered together and the important terms that were found common among the documents in the cluster. Check out the cluster output.


In [10]:
print("**** TOP 10 TERMS PER CLUSTER ****")
orderedCentroids = km.cluster_centers_.argsort()[:, ::-1] # get indexes - sort smallest to largest then reverse order, largest to smallest
orderedCentroidScores = []
for i in range(numClusters):
    orderedCentroidScores.append(km.cluster_centers_[i, orderedCentroids[i,:]])  
for i in range(numClusters):
    print("Cluster# %d Words:" % i, end='')
    j = 0
    for indice in orderedCentroids[i, :10]:
        print(' %s: %.5f' % (distinctTerms[indice].split(' ')[0], orderedCentroidScores[i][j]), end=',')
        j += 1
    print()
    print()
    print("Cluster# %d Titles:" % i, end='')
    for label in frame.ix[i]['labels'].values.tolist():
        print(' %s,' % label, end='')
    print()
    print()


**** TOP 10 TERMS PER CLUSTER ****
Cluster# 0 Words: left: 0.15677, effusion: 0.15308, chest: 0.14856, right: 0.14209, lobe: 0.13176, pleural: 0.13027, pleural: 0.12160, lower: 0.11573, lower: 0.11279, ap: 0.09157,

Cluster# 0 Titles: PNA-POS1, PNA-POS2, PNA-POS7, PNA-POS8, PNA-POS23, PNA-POS25, PNA-POS26, PNA-POS33, PNA-POS34, PNA-POS35, PNA-POS43, PNA-POS67, PNA-POS68, PNA-POS77, PNA-POS79, PNA-POS85, PNA-POS96, PNA-POS99, PNA-POS100, PNA-POS105, PNA-POS112, PNA-POS113, PNA-POS115, PNA-POS117, PNA-POS124, PNA-POS125, PNA-POS132, PNA-POS134, PNA-POS137, PNA-POS138, PNA-POS139, PNA-POS141, PNA-POS147, PNA-POS152, PNA-POS153, PNA-POS156, PNA-POS161, PNA-POS165, PNA-POS167, PNA-POS168, PNA-POS170, PNA-POS171, PNA-POS173, PNA-POS174, PNA-POS175, PNA-POS177, PNA-POS181, PNA-POS183, PNA-POS184, PNA-POS188, PNA-POS189, PNA-POS191, PNA-POS193, PNA-POS197, PNA-POS200, PNA-POS201, PNA-POS205, PNA-POS207, PNA-POS209, PNA-POS210, PNA-POS211, PNA-POS221, PNA-POS222, PNA-POS227, PNA-POS228, PNA-POS

/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:15: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  from ipykernel import kernelapp as app


 PNA-NEG5810, PNA-NEG5816, PNA-NEG5823, PNA-NEG5824, PNA-NEG5825, PNA-NEG5826, PNA-NEG5830, PNA-NEG5831, PNA-NEG5832, PNA-NEG5833, PNA-NEG5834, PNA-NEG5836, PNA-NEG5837, PNA-NEG5838, PNA-NEG5839, PNA-NEG5840, PNA-NEG5877, PNA-NEG5882, PNA-NEG5900, PNA-NEG5904, PNA-NEG5905, PNA-NEG5965, PNA-NEG6003, PNA-NEG6028, PNA-NEG6029, PNA-NEG6036, PNA-NEG6037, PNA-NEG6038, PNA-NEG6039, PNA-NEG6040, PNA-NEG6048, PNA-NEG6069, PNA-NEG6071, PNA-NEG6073, PNA-NEG6077, PNA-NEG6078, PNA-NEG6081, PNA-NEG6082, PNA-NEG6083, PNA-NEG6084, PNA-NEG6089, PNA-NEG6092, PNA-NEG6093, PNA-NEG6094, PNA-NEG6101, PNA-NEG6102, PNA-NEG6246, PNA-NEG6257, PNA-NEG6259, PNA-NEG6262, PNA-NEG6314, PNA-NEG6316, PNA-NEG6317, PNA-NEG6318, PNA-NEG6326, PNA-NEG6327, PNA-NEG6329, PNA-NEG6331, PNA-NEG6344, PNA-NEG6365, PNA-NEG6370, PNA-NEG6375, PNA-NEG6378, PNA-NEG6386, PNA-NEG6388, PNA-NEG6390, PNA-NEG6396, PNA-NEG6421, PNA-NEG6453, PNA-NEG6467, PNA-NEG6474, PNA-NEG6497, PNA-NEG6518, PNA-NEG6567, PNA-NEG6577, PNA-NEG6589, PNA-NEG6600

Cluster# 2 Titles: PNA-POS4, PNA-POS5, PNA-POS6, PNA-POS13, PNA-POS14, PNA-POS15, PNA-POS17, PNA-POS18, PNA-POS19, PNA-POS44, PNA-POS45, PNA-POS46, PNA-POS47, PNA-POS51, PNA-POS52, PNA-POS55, PNA-POS56, PNA-POS58, PNA-POS59, PNA-POS60, PNA-POS74, PNA-POS75, PNA-POS76, PNA-POS107, PNA-POS108, PNA-POS109, PNA-POS122, PNA-POS123, PNA-POS126, PNA-POS129, PNA-POS130, PNA-POS131, PNA-POS145, PNA-POS146, PNA-POS162, PNA-POS163, PNA-POS164, PNA-POS212, PNA-POS213, PNA-POS256, PNA-POS257, PNA-POS258, PNA-POS259, PNA-POS291, PNA-POS293, PNA-POS294, PNA-POS315, PNA-POS316, PNA-POS317, PNA-POS348, PNA-POS349, PNA-POS350, PNA-POS374, PNA-POS375, PNA-POS388, PNA-POS389, PNA-POS421, PNA-POS422, PNA-POS423, PNA-POS439, PNA-POS440, PNA-POS441, PNA-POS515, PNA-POS516, PNA-POS523, PNA-POS524, PNA-POS525, PNA-POS548, PNA-POS549, PNA-POS550, PNA-POS565, PNA-POS566, PNA-POS567, PNA-POS568, PNA-POS579, PNA-POS580, PNA-POS581, PNA-POS582, PNA-POS622, PNA-POS623, PNA-POS624, PNA-POS704, PNA-POS705, PNA-POS706,

Cluster# 4 Titles: PNA-POS340, PNA-POS424, PNA-POS480, PNA-POS492, PNA-POS618, PNA-POS619, PNA-POS620, PNA-POS656, PNA-POS660, PNA-POS717, PNA-POS733, PNA-POS739, PNA-POS772, PNA-POS795, PNA-POS814, PNA-POS842, PNA-POS865, PNA-POS870, PNA-POS879, PNA-POS880, PNA-POS882, PNA-POS883, PNA-POS884, PNA-POS898, PNA-POS900, PNA-POS912, PNA-POS923, PNA-POS931, PNA-POS970, PNA-POS987, PNA-POS994, PNA-POS998, PNA-POS999, PNA-POS1004, PNA-POS1005, PNA-POS1006, PNA-POS1019, PNA-POS1020, PNA-POS1028, PNA-POS1031, PNA-POS1032, PNA-POS1036, PNA-POS1042, PNA-POS1043, PNA-POS1044, PNA-POS1061, PNA-POS1063, PNA-POS1073, PNA-POS1083, PNA-POS1086, PNA-POS1093, PNA-POS1098, PNA-NEG14, PNA-NEG15, PNA-NEG31, PNA-NEG58, PNA-NEG66, PNA-NEG67, PNA-NEG68, PNA-NEG89, PNA-NEG93, PNA-NEG101, PNA-NEG102, PNA-NEG109, PNA-NEG111, PNA-NEG126, PNA-NEG130, PNA-NEG131, PNA-NEG132, PNA-NEG161, PNA-NEG162, PNA-NEG175, PNA-NEG237, PNA-NEG240, PNA-NEG243, PNA-NEG246, PNA-NEG250, PNA-NEG251, PNA-NEG252, PNA-NEG259, PNA-NEG260,

Cluster# 5 Titles: PNA-POS887, PNA-POS888, PNA-POS889, PNA-NEG127, PNA-NEG149, PNA-NEG150, PNA-NEG151, PNA-NEG152, PNA-NEG153, PNA-NEG154, PNA-NEG155, PNA-NEG156, PNA-NEG157, PNA-NEG158, PNA-NEG159, PNA-NEG160, PNA-NEG163, PNA-NEG164, PNA-NEG165, PNA-NEG166, PNA-NEG167, PNA-NEG168, PNA-NEG169, PNA-NEG170, PNA-NEG171, PNA-NEG172, PNA-NEG173, PNA-NEG174, PNA-NEG176, PNA-NEG177, PNA-NEG178, PNA-NEG179, PNA-NEG180, PNA-NEG181, PNA-NEG281, PNA-NEG642, PNA-NEG854, PNA-NEG855, PNA-NEG1058, PNA-NEG1060, PNA-NEG1061, PNA-NEG1062, PNA-NEG1063, PNA-NEG1064, PNA-NEG1065, PNA-NEG1066, PNA-NEG1067, PNA-NEG1068, PNA-NEG1069, PNA-NEG1182, PNA-NEG1183, PNA-NEG1184, PNA-NEG1188, PNA-NEG1189, PNA-NEG1190, PNA-NEG1191, PNA-NEG1201, PNA-NEG1202, PNA-NEG1203, PNA-NEG1204, PNA-NEG1205, PNA-NEG1206, PNA-NEG1207, PNA-NEG1208, PNA-NEG1209, PNA-NEG1210, PNA-NEG1503, PNA-NEG1541, PNA-NEG1542, PNA-NEG1543, PNA-NEG1544, PNA-NEG1545, PNA-NEG1546, PNA-NEG1547, PNA-NEG1548, PNA-NEG1549, PNA-NEG1584, PNA-NEG1585, PNA-N

Cluster# 8 Titles: PNA-POS3, PNA-POS9, PNA-POS10, PNA-POS12, PNA-POS16, PNA-POS20, PNA-POS21, PNA-POS22, PNA-POS27, PNA-POS28, PNA-POS30, PNA-POS32, PNA-POS36, PNA-POS38, PNA-POS39, PNA-POS40, PNA-POS41, PNA-POS49, PNA-POS50, PNA-POS53, PNA-POS54, PNA-POS61, PNA-POS62, PNA-POS63, PNA-POS64, PNA-POS69, PNA-POS70, PNA-POS71, PNA-POS78, PNA-POS83, PNA-POS84, PNA-POS86, PNA-POS87, PNA-POS88, PNA-POS89, PNA-POS90, PNA-POS91, PNA-POS92, PNA-POS93, PNA-POS114, PNA-POS118, PNA-POS119, PNA-POS148, PNA-POS149, PNA-POS150, PNA-POS151, PNA-POS158, PNA-POS172, PNA-POS176, PNA-POS178, PNA-POS194, PNA-POS195, PNA-POS196, PNA-POS206, PNA-POS217, PNA-POS218, PNA-POS287, PNA-POS323, PNA-POS328, PNA-POS329, PNA-POS346, PNA-POS347, PNA-POS358, PNA-POS359, PNA-POS538, PNA-POS539, PNA-POS593, PNA-POS594, PNA-POS640, PNA-POS641, PNA-POS648, PNA-POS649, PNA-POS702, PNA-POS703, PNA-POS737, PNA-POS738, PNA-POS740, PNA-POS742, PNA-POS743, PNA-POS754, PNA-POS755, PNA-POS765, PNA-POS768, PNA-POS769, PNA-POS770, PN

Cluster# 9 Titles: PNA-POS854, PNA-POS855, PNA-POS856, PNA-POS914, PNA-POS915, PNA-POS916, PNA-POS917, PNA-POS918, PNA-POS919, PNA-POS920, PNA-POS1025, PNA-POS1026, PNA-POS1027, PNA-POS1055, PNA-POS1056, PNA-POS1057, PNA-POS1058, PNA-POS1059, PNA-POS1068, PNA-POS1069, PNA-POS1070, PNA-POS1087, PNA-POS1088, PNA-POS1089, PNA-POS1090, PNA-POS1091, PNA-NEG82, PNA-NEG83, PNA-NEG84, PNA-NEG85, PNA-NEG86, PNA-NEG87, PNA-NEG88, PNA-NEG186, PNA-NEG187, PNA-NEG188, PNA-NEG229, PNA-NEG230, PNA-NEG231, PNA-NEG232, PNA-NEG233, PNA-NEG234, PNA-NEG235, PNA-NEG244, PNA-NEG245, PNA-NEG247, PNA-NEG256, PNA-NEG257, PNA-NEG258, PNA-NEG354, PNA-NEG357, PNA-NEG421, PNA-NEG425, PNA-NEG434, PNA-NEG435, PNA-NEG436, PNA-NEG581, PNA-NEG582, PNA-NEG583, PNA-NEG584, PNA-NEG600, PNA-NEG601, PNA-NEG602, PNA-NEG640, PNA-NEG641, PNA-NEG787, PNA-NEG796, PNA-NEG797, PNA-NEG870, PNA-NEG871, PNA-NEG873, PNA-NEG874, PNA-NEG900, PNA-NEG901, PNA-NEG902, PNA-NEG1028, PNA-NEG1029, PNA-NEG1030, PNA-NEG1109, PNA-NEG1110, PNA-NEG

## Cluster Visualization

#### We have a bit of a problem in that we can only conceptualize what high-dimensional space must look like. We are only able to visualize 3-dimensional space and most of us prefer 2-dimensional visualization. With text clustering we are dealing with 100's of dimensions, one for each vector term we used. Well the good news is that we have figured out how to perform reduction mappiing which projects high-dimensional space onto lower 2-dimensional space for visualizetion. We will use a process known as multidemensional scaling that attemps to preserve the distance between objects when reduced to low dimensional space. This may take a little time so be patient, eventually you will see the clusters in 2-dimensional space.    

In [ ]:
#plt.switch_backend('agg')
# Multidimensional scaling (MDS) is a means of visualizing the level of similarity of individual cases in a dataset. 
# MDS aims to place each object in n-dimensional space such that the between-object distances are preserved
# in the reduction mapping to two-dimensional space.
MDS()
# "precomputed" because we provide a distance matrix
# "random_state" so the plot is reproducible
print("Initialize MDS")
mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)
print("Fit MDS")
pos = mds.fit_transform(dist)
print("MDS fit complete")

xs, ys = pos[:, 0], pos[:, 1]
#set up colors for each clusters
cluster_colors = {0: '#1b9e77', 1: '#d95f02', 2: '#7570b3', 3: '#e7298a', 4: '#66a61e', 
                  5: '#ffcc99', 6: '#33ffcc', 7: '#FFFF33', 8: '#ccccff', 9: '#99ccff'}
#set up cluster names for each color
cluster_names = {0: 'Cluster 0', 1: 'Cluster 1', 2: 'Cluster 2', 3: 'Cluster 3', 4: 'Cluster 4', 
                 5: 'Cluster 5', 6: 'Cluster 6', 7: 'Cluster 7', 8: 'Cluster 8', 9: 'Cluster 9'} 
#result of the MDS along with the cluster numbers and titles
df = pd.DataFrame(dict(x=xs, y=ys, label=clusters, title=labels)) 
#group by cluster
groups = df.groupby('label')
# set up plot
fig, ax = plt.subplots(figsize=(17, 9)) # set size
ax.margins(0.05)
#iterate through groups to layer the plot
#note that I use the cluster_name and cluster_color dicts with the 'name' lookup to return the appropriate color/label
for name, group in groups:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, label=cluster_names[name], color=cluster_colors[name], mec='none')
    ax.set_aspect('auto')
    ax.tick_params(\
        axis= 'x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelbottom='off')
    ax.tick_params(\
        axis= 'y',         # changes apply to the y-axis
        which='both',      # both major and minor ticks are affected
        left='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelleft='off')   
ax.legend(numpoints=1)  #show legend with only 1 point
#add label in x,y position with the label as the document name
for i in range(len(df)):
    ax.text(df.ix[i]['x'], df.ix[i]['y'], df.ix[i]['title'], size=8)  
plt.show() #show the plot

Initialize MDS
Fit MDS
